In [ ]:
import numpy as np
import os
import pathlib
import skimage.io
import skimage.transform
import tensorflow

from metrics import dice_coef, dice_loss, jaccard_distance

In [ ]:
image_size = (400, 400)
path_imgs = os.path.join('../dataset_george', 'imgs', 'RGB', '400', 'OUT')
out_folder = os.path.join('../dataset_george', 'imgs_segmentada', 'RGB', '400')
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

list_images = sorted([file for file in pathlib.Path(path_imgs).rglob('*')])

os.path.exists(path_imgs), len(list_images)

In [ ]:
for d in ['mask_unet', 'w_pred_mask']:
    p = os.path.join(out_folder, d)
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

model = tensorflow.keras.models.load_model('predict/400/unet.h5', custom_objects = {'dice_loss': dice_loss, 'dice_coef': dice_coef, 'jaccard_distance': jaccard_distance })

for i, file in enumerate(list_images):
    print(i, file.resolve())
    img = skimage.io.imread(file.resolve())
    img = skimage.transform.resize(img, image_size)
    img = img.reshape((1, 400, 400, 3))
    mask = model.predict(img)
    mask = mask[0, :, :, :]
    new_filename = os.path.join(out_folder, 'mask_unet', str(file.stem) + '.png')
    skimage.io.imsave(new_filename, skimage.img_as_ubyte(mask))

    image_segmented = img * mask
    image_segmented[image_segmented == 0] = 1
    image_segmented = image_segmented[0, :, :, :]
    new_filename = os.path.join(out_folder, 'w_pred_mask', str(file.stem) + '.png')
    skimage.io.imsave(new_filename, skimage.img_as_ubyte(image_segmented))